In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from time import time

In [2]:
# Model parameters
CLASS_NUM = 2

IMAGE_HEIGHT = 180
IMAGE_WIDTH = 180

data_dir = '../dataset'

In [3]:
# Load dataset
Transforms = transforms.Compose([
    transforms.Resize((IMAGE_HEIGHT, IMAGE_WIDTH)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.Grayscale(num_output_channels=1)
])

dataset = torchvision.datasets.ImageFolder(
    root=data_dir,
    transform=Transforms
)

In [4]:
t0 = time()
data = []
for (image, label) in dataset:
    x = torch.flatten(image, 1)
    x = x[0].detach().numpy()
    data.append(x)
labels = dataset.targets
print("[INFO] LOADED", len(labels), "IMAGES in %0.3fs" % (time() - t0))

C:\ProgramData\Anaconda3\envs\Pytorch\lib\site-packages\PIL\Image.py:951: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


[INFO] LOADED 3400 IMAGES in 6.660s


In [5]:
# SPLIT DATA INTO TRAINING AND TESTING SET
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    data, labels,
    test_size=0.20,
    stratify=labels
)

In [6]:
from sklearn.metrics import classification_report
def EVALUATE(clf, clf_name):
    print("[",clf_name,"]")
    print("classification report on the test set")
    t0 = time()
    y_pred = clf.predict(X_test)
    print("done in %0.3fs" % (time() - t0))
    print(classification_report(y_test, y_pred))

In [15]:
from sklearn.ensemble import RandomForestClassifier
RandomForest = RandomForestClassifier(criterion='entropy', bootstrap=False, n_jobs=-1)
RandomForest.fit(X_train, y_train)
EVALUATE(RandomForest, "Random Forest")

[ Random Forest ]
classification report on the test set
done in 0.061s
              precision    recall  f1-score   support

           0       0.92      0.84      0.88       294
           1       0.88      0.95      0.91       386

    accuracy                           0.90       680
   macro avg       0.90      0.89      0.90       680
weighted avg       0.90      0.90      0.90       680



In [16]:
from sklearn.model_selection import cross_val_score
from sklearn import metrics
scores = cross_val_score(RandomForest, data, labels, cv=5, n_jobs=-1)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.86 (+/- 0.18)
